Program that, given a list and location of images that have the planet center and north pole position angle saved in the header, rotates and centers the images and saves them into a single FITS file

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
import math
import scipy
from scipy import ndimage 
# info on ndimage: https://docs.scipy.org/doc/scipy/reference/generated/scipy.ndimage.rotate.html

In [ ]:
directory = '/Volumes/external_hd/march2017/redo/jupiter_2/' # location of images
imlist = np.loadtxt('/Volumes/external_hd/march2017/march_2017_sciimages',dtype=str) # list of images
image_cube_name = '/Users/dahlek/Desktop/march2017_cube.fits' # name of final image cube fits file

length = 600 # size of square array that image is fit onto
croplength = 140 # length to crop off final image
starting_index = 0
data_list = []

for i in range(starting_index,len(imlist)):
    im = fits.open(directory+imlist[i])
    if im[0].header['rfon'] == 1:
        print imlist[i], im[0].header['lambda'],im[0].header['pnorth']
        pnorth = im[0].header['pnorth']
        pceny = im[0].header['pceny']
        pcenx = im[0].header['pcenx']
        # make a big blank array
        blank = np.zeros((length,length))
        blankcenx = length/2
        blankceny = length/2
        diffx = blankcenx-pcenx
        diffy = blankceny-pceny
        # map jupiter image onto blank array
        for f in range(0,len(blank)):
            for k in range(0,len(blank[f])):
                if f < im[0].header['NAXIS1']:
                    if k < im[0].header['NAXIS2']:
                        blank[int(f+diffy)][int(k+diffx)] = im[0].data[f][k]
        
        length2 = len(scipy.ndimage.rotate(blank,-pnorth))

        # plot rotated and centered image; uncomment if you want to see/check each image
        '''plt.matshow(np.flip(scipy.ndimage.rotate(blank,-pnorth),axis=1) [croplength:(length2-croplength),croplength:(length2-croplength)])
        # https://docs.scipy.org/doc/numpy/reference/generated/numpy.flip.html
        # plot crosshairs
        plt.plot([(length2-2*croplength)/2,(length2-2*croplength)/2],[0,(length2-2*croplength)],'red')
        plt.plot([0,(length2-2*croplength)],[(length2-2*croplength)/2,(length2-2*croplength)/2],'red')
        plt.xlim(0,length2-2*croplength)
        plt.ylim(0,length2-2*croplength)
        plt.show()'''
        
        # put it into a fits file
        data_list.append(np.flip(scipy.ndimage.rotate(blank,-pnorth),axis=1) [croplength:(length2-croplength),croplength:(length2-croplength)]) # rotated, flipped, cropped jupiter image

    im.close()

# after looping though images, save them to a fits file
hdu = fits.HDUList(fits.PrimaryHDU(data_list))
print hdu.info()
firstwl = fits.open(directory+imlist[starting_index])[0].header['lambda']
hdu[0].header['FIRST_WL'] = (firstwl,'First wavelength')
hdu.writeto(image_cube_name,overwrite = True)